<a href="https://colab.research.google.com/github/max-ostapenko/ga4_data_import/blob/main/scripts/GCS_to_GA4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 1 Start with documentation
Read [the official documentation](https://support.google.com/analytics/answer/10071301) to have an idea about features and limitations of GA4 data import.


## 2 Create a Data Source in [GA4 Admin Panel](https://analytics.google.com/analytics/web/#/admin/data-import/)

0. Follow [the guide on configuring SFTP data source](https://support.google.com/analytics/answer/10071301#import)

In [1]:
# @markdown 1. Get your SFTP server url by deploying VM Instance
import os

GCP_PROJECT_ID = 'max-ostapenko'  #@param {type: "string"}
ZONE = 'us-central1-a' #@param {type: "string"}
INSTANCE_NAME = 'sftp-server' #@param {type: "string"}
SFTP_USERNAME = 'sftp_user' #@param {type: "string"}

!gcloud config set project {GCP_PROJECT_ID}
if os.getenv("COLAB_RELEASE_TAG"):
    !pip install git+https://github.com/max-ostapenko/ga4_data_import.git
    from google.colab import auth
    auth.authenticate_user()
#else:
    #!gcloud auth application-default login

from ga4_data_import.compute import create_static_address, create_instance
from ga4_data_import.storage import add_bucket_read_access



#@markdown 2. Enter the name of your Cloud Storage Bucket to mount it as read-only on SFTP server
BUCKET_NAME = 'sftp-server-ga4' #@param {type: "string"}

INSTANCE_IP = "35.208.180.229"  # create_static_address(GCP_PROJECT_ID, ZONE, INSTANCE_NAME)


create_instance(
    project_id=GCP_PROJECT_ID,
    zone=ZONE,
    instance_name=INSTANCE_NAME,
    static_address=INSTANCE_IP,
    sftp_username=SFTP_USERNAME,
    bucket_name=BUCKET_NAME
)


# Give READ access to a bucket to default VM Service Account
add_bucket_read_access(GCP_PROJECT_ID, BUCKET_NAME)

print(f"""\n
Now you can enter your SFTP server url to Data Import UI:
    sftp://{INSTANCE_IP}/cost_data.csv
where `cost_data.csv` - name of a file in the bucket root""")

Updated property [core/project].


Now you can enter your SFTP server url to Data Import UI:
- simple: sftp://35.208.180.229/cost_data.csv

where `cost_data.csv` - name of a file in the bucket root


In [2]:
from ga4_data_import.compute import add_shh_pub_key

# @markdown 3. Paste and push Public Key from GA4 to SFTP server for authorisation
KEY_VALUE = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAABgQDeDrtHfPb8TM6/BqANwQJdSmAGjygLE+2jMdIxfJZFRdcEogMLMrEARhzj9fjRQKw7RhudOpxkyvMjVRXlv6zvcbvmJhQCxk6yp3Cgq3U7ux2PYppBWSxei1R4DH2TSjy/k/7nuKDvdDaBwUC/WLOgjvQ5nKLbYp53RbWoPrv63YEDDEyuLjMww9rlJ+J9iL/fd1TxlZxvwVvPCxweWCzfQfLP6MWGvDmUMfe7eTg11rt3p/wdNUa/RxKC1HOJRWloGCw7dX5LgZYoCq8tKOS8rpHt8BoTFhKjoPsIw8h4uNsjOb9y6YSOQrQFaXj3o2l4X0ssLxkKQcaG+EDZ8PtUr/WnuPpfHCuTNN1Q6TnaF2t2zm4J7C6LchPMgiP/vO+wwloQpgg9tUVGC4FZxQhHTLmCoW+19KIuBu4nm+zgaeIWXuHfnPLE4UdFsIB3o76tAfZprrykjaWVLPc6PFVlvRx1PIAPTL7dHaajnv1rxuQdQlNU2a2dOE4KNoxnffE= Google Analytics Data Import Key"  # @param {type: "string"}
add_shh_pub_key(GCP_PROJECT_ID, ZONE, INSTANCE_NAME, SFTP_USERNAME, KEY_VALUE)

LOCAL_KEY = "ssh-rsa AAAAB3NzaC1yc2EAAAADAQABAAACAQDIjG9uMbjXCMmJ6oLlI5rVlbm+eY2Ur42fCE542NAGwR64lEYmv2LkxPRzFnVU2mdF6+Ka4D8UyjM6D3fVkNfqQH8ckthcoMEgCNNK6tSU690I5YXAfGvCstGnljG4MwUWHkCFGXntqHPVnWjRfe0M88CusqazLy06VOr5hdfjTUXyxbyy4vx3kzBXE2oBXl0UZ33fI0/jHG3u6PULZ9kYu8jCXnNGp2b3ntgsMhitWTfj4GiSZF5wf6t4qDupgaW6OCrLJkgq9WS4OFdM4ZyHJadRff7KyHjsjtFoXzHOxhCrk4gj484HO86QyoA8jJE92o6k7OuVP9/Dl/w5n2jRtsvj0Fx2UJM3LH57XvflWWKOvfoYXl1v0Li78xnzLIgwDi282CHh31NNlbdm0Ux8AQXuQlLWd4R13sleLmdpqpcvrkCHktpvOIOvndar311CsPW3AvnHxNOKNAaJfE1UcP45AJ5TU6DrRJj9gAi+AJ/iET9W5XtKJOIk8JtHX6TwCXlSloZgPf+LOOscnYb5EQO93LCSyHQVwXuF35hV9DPFTXM+PUdhZCaPLwETjnoP8pQMYg1h8B6hCY9cXWlZQpxDyx4TUP/8vRuizX2oR5rLDG7p0P8x+IX6NtnuCQNockzl6DqXup/4o8SNVPHbC7Dv9uD9/zm9Pz7BK8TA+Q=="
add_shh_pub_key(GCP_PROJECT_ID, ZONE, INSTANCE_NAME, SFTP_USERNAME, LOCAL_KEY)


## 3 Test your SFTP server connention

1. Open [Google Cloud Shell terminal](https://shell.cloud.google.com/)

2. Authorise local SSH Key on the server
2. Connect to SFTP server
3. Show available files

In [3]:
#@markdown Press RUN
import os
from ga4_data_import.common import read_pub_key
from ga4_data_import.compute import add_shh_pub_key

ssh_key_file = os.path.expanduser("~/.ssh/id_rsa.pub")
if not os.path.isfile(ssh_key_file):
    !ssh-keygen -t rsa -b 4096 -f ~/.ssh/id_rsa -N '' -q
colab_sshpubkey = read_pub_key(ssh_key_file)

add_shh_pub_key(GCP_PROJECT_ID, ZONE, INSTANCE_NAME, SFTP_USERNAME, colab_sshpubkey + " Developer's Key")
!ssh-keygen -R {INSTANCE_IP}
!sftp {SFTP_USERNAME}@{INSTANCE_IP} -y

Host 35.208.180.229 not found in /home/vscode/.ssh/known_hosts
The authenticity of host '35.208.180.229 (35.208.180.229)' can't be established.
ECDSA key fingerprint is SHA256:HBhfakafhoyr9ZUfvKuwxPieYlnhNNHRQcZhs+Gj2Kw.
Are you sure you want to continue connecting (yes/no/[fingerprint])? ^C
Host key verification failed.


In [1]:
!pip install git+https://github.com/max-ostapenko/ga4_data_import.git

Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/max-ostapenko/ga4_data_import.git to /tmp/pip-req-build-2e2dmy60
  Running command git clone --filter=blob:none --quiet https://github.com/max-ostapenko/ga4_data_import.git /tmp/pip-req-build-2e2dmy60
  Resolved https://github.com/max-ostapenko/ga4_data_import.git to commit 91ece0e6723f5cebf58846b957f0b410fc88427c
  Preparing metadata (setup.py) ... done
  Created wheel for ga4-data-import: filename=ga4_data_import-0.1.0-py3-none-any.whl size=13461 sha256=c723bfa2ef2e7c40b62a89a45249d2407ff84b9c73c8661f72f8700c093570ce
  Stored in directory: /tmp/pip-ephem-wheel-cache-3s5_o1ny/wheels/c7/f6/37/9237e077574970fbb70fb499d00c5de5bcf94b7fb6975e0967
Successfully built ga4-data-import
